In [27]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

#text completion(완성) 할 때 유용하다. 예측을 해야할 때, 텍스트를 자동완성하고 싶을 때 
#모든 메모리는 save_context, load_memory_variables 함수를 갖고 있다. 메모리 종류만 다를 뿐 API는 같다 

{'history': 'Human: Hi!\nAI: How are you?'}